In [1]:
# %tensorflow_version 1.x
from time import time
from scipy.sparse import csc_matrix
import tensorflow as tf
import numpy as np
import h5py

In [3]:
def load_data_100k(path='./', delimiter='\t'):

    train = np.loadtxt(path+'movielens_100k_u1.base', skiprows=0, delimiter=delimiter).astype('int32')
    test = np.loadtxt(path+'movielens_100k_u1.test', skiprows=0, delimiter=delimiter).astype('int32')
    total = np.concatenate((train, test), axis=0)

    n_u = np.unique(total[:,0]).size  # num of users
    n_m = np.unique(total[:,1]).size  # num of movies
    n_train = train.shape[0]  # num of training ratings
    n_test = test.shape[0]  # num of test ratings

    train_r = np.zeros((n_m, n_u), dtype='float32')
    test_r = np.zeros((n_m, n_u), dtype='float32')

    for i in range(n_train):
        train_r[train[i,1]-1, train[i,0]-1] = train[i,2]

    for i in range(n_test):
        test_r[test[i,1]-1, test[i,0]-1] = test[i,2]

    train_m = np.greater(train_r, 1e-12).astype('float32')  # masks indicating non-zero entries
    test_m = np.greater(test_r, 1e-12).astype('float32')

    print('data matrix loaded')
    print('num of users: {}'.format(n_u))
    print('num of movies: {}'.format(n_m))
    print('num of training ratings: {}'.format(n_train))
    print('num of test ratings: {}'.format(n_test))

    return n_m, n_u, train_r, train_m, test_r, test_m

In [4]:
# Common hyperparameter settings
n_hid = 500
n_dim = 5
n_layers = 2
gk_size = 3

lambda_2 = 20.  # l2 regularisation
lambda_s = 0.006
iter_p = 5  # optimisation
iter_f = 5
epoch_p = 30  # training epoch
epoch_f = 60
dot_scale = 1  # scaled dot product

In [5]:
# Insert the path of a data directory by yourself (e.g., '/content/.../data')
# .-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
data_path = 'datas/'
# .-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._

In [6]:
path = data_path + '/MovieLens_100K/'
n_m, n_u, train_r, train_m, test_r, test_m = load_data_100k(path=path, delimiter='\t')

data matrix loaded
num of users: 943
num of movies: 1682
num of training ratings: 80000
num of test ratings: 20000


In [7]:
def local_kernel(u, v):
    dist = tf.norm(u - v, ord=2, axis=2)
    hat = tf.maximum(0., 1. - dist**2)

    return hat

In [37]:
#AE Model
n_in = n_u
n_dim = 5
n_hid = 500
lambda_2 = 20.  # l2 regularisation
lambda_s = 0.006

# y = tf.compat.v1.placeholder("float", [n_m, n_u])
y = tf.Variable(tf.ones(shape=[n_m, n_u], dtype=float))

default_initializer = tf.keras.initializers.GlorotUniform()

W = tf.Variable(default_initializer(shape=[n_in, n_hid]), name='W')
u = tf.Variable(tf.random.truncated_normal([n_in, 1, n_dim]), name="u")
v = tf.Variable(tf.random.truncated_normal([1, n_hid, n_dim]), name="v")
b = tf.Variable(default_initializer(shape=[n_hid]))

w_hat = local_kernel(u, v)

sparse_reg = tf.keras.regularizers.L2(lambda_s)
sparse_reg_term = sparse_reg(w_hat)

l2_reg = tf.keras.regularizers.L2(lambda_2)
l2_reg_term = l2_reg(W)

loss_value = sparse_reg_term + l2_reg_term
print(sparse_reg_term, l2_reg_term, loss_value)

W_eff = W * w_hat
print(W_eff)

y = tf.matmul(y, W_eff) + b
y = tf.nn.sigmoid(y)

print(y)

tf.Tensor(4.4011245, shape=(), dtype=float32) tf.Tensor(13054.93, shape=(), dtype=float32) tf.Tensor(13059.331, shape=(), dtype=float32)
tf.Tensor(
[[ 0.          0.          0.         ...  0.         -0.
  -0.        ]
 [-0.         -0.         -0.         ... -0.         -0.
  -0.        ]
 [-0.         -0.         -0.         ...  0.          0.
  -0.        ]
 ...
 [ 0.          0.          0.         ... -0.         -0.
   0.02342543]
 [-0.         -0.          0.         ...  0.         -0.
   0.        ]
 [-0.          0.         -0.         ...  0.          0.
   0.        ]], shape=(943, 500), dtype=float32)
tf.Tensor(
[[0.47870147 0.49157372 0.51399034 ... 0.4814304  0.5237499  0.50520736]
 [0.47870147 0.49157372 0.51399034 ... 0.4814304  0.5237499  0.50520736]
 [0.47870147 0.49157372 0.51399034 ... 0.4814304  0.5237499  0.50520736]
 ...
 [0.47870147 0.49157372 0.51399034 ... 0.4814304  0.5237499  0.50520736]
 [0.47870147 0.49157372 0.51399034 ... 0.4814304  0.5237499  0.505

In [122]:
def local_kernel(u, v):
    dist = tf.norm(u - v, ord=2, axis=2)
    hat = tf.maximum(0., 1. - dist**2)

    return hat

class kernel_layer(tf.keras.layers.Layer):
    def __init__(self, n_hid=500, activation=tf.nn.sigmoid):
        super(kernel_layer, self).__init__()
        
        self.n_dim = 5
        self.n_hid = n_hid
        self.lambda_2 = 20.  # l2 regularisation
        self.lambda_s = 0.006
        
        self.activation = activation
        
    def call(self, inputs):
        default_initializer = tf.keras.initializers.GlorotUniform()

        W = tf.Variable(default_initializer(shape=[inputs.shape[1], self.n_hid]), name='W')
        n_in = inputs.shape[1]
        u = tf.Variable(tf.random.truncated_normal([n_in, 1, self.n_dim]), name="u")
        v = tf.Variable(tf.random.truncated_normal([1, self.n_hid, self.n_dim]), name="v")
        b = tf.Variable(default_initializer(shape=[self.n_hid]))
        
        w_hat = local_kernel(u, v)

        sparse_reg = tf.keras.regularizers.L2(self.lambda_s)
        sparse_reg_term = sparse_reg(w_hat)

        l2_reg = tf.keras.regularizers.L2(self.lambda_2)
        l2_reg_term = l2_reg(W)

        loss_value = sparse_reg_term + l2_reg_term

        W_eff = W * w_hat

        y = tf.matmul(inputs, W_eff) + b
        y = self.activation(y)

        return y, loss_value

In [134]:
y = tf.Variable(tf.ones(shape=[n_m, n_u], dtype=float))
reg_losses = None

kl = kernel_layer()

for i in range(n_layers):
    y, reg_loss = kl(y)
    reg_losses = reg_loss if reg_losses is None else reg_losses + reg_loss
    

kl2 = kernel_layer(n_u, activation=tf.identity)
pred_p, reg_loss = kl2(y)
reg_losses += reg_loss

# L2 Loss
diff = train_m * (train_r - pred_p)
sqE = tf.nn.l2_loss(diff)
loss_p = sqE + reg_losses

optimizer = tf.optimizers.SGD()



In [135]:
import tensorflow_probability as tfp

ModuleNotFoundError: No module named 'tensorflow_probability'

In [47]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)